<a href="https://colab.research.google.com/github/dnaneet/ComputationalMechanicsPlayGround/blob/master/MonteCarlo_TorsionOfCircularShafts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from numba import jit
import random
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px 
import timeit
from time import process_time 
import pandas as pd

# Monte Carlo simulations

Monte Carlo simulations are a fancy way of saying that repeated random sampling is performed to generate a large number of numerical quantities.  This is not dissimilar to gambling at the casinos of Monte Carlo and consequently, this method is named so.  MC simulations are used in a wide variety of high-impact situations in society: stock/share price estimation, ensemble simulations of weather patterns, reliability engineering for machine component design to name a few.

## Monte Carlo estimation of $\pi$

In [0]:
'''
Estimating the value of PI using Monte Carlo simulation:
Define equation of a circle of radius 1:
x^2 + y^2 = 1 ...[I]

Use uniformly distributed random sample values of x and y and plug these
values into equation [I].  For every pair sample,
if x^2 + y^2 < 1: this pair falls inside the boundary of the circle.
Every time a pair of samples falls inside the boundary of the circle, 
increment a counter by 1.   


4*Total number of samples/samples that fall within the circle = PI.

'''

@jit(nopython=True)
def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

def monte_carlo_pi_unoptimized(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [0]:
nSamp=1000

t1_start = process_time()  
#monte_carlo_pi_unoptimized(nSamp) #do not try this if nSamp > 1e4
t1_end = process_time()

t2_start = process_time()
print(monte_carlo_pi(nSamp))
t2_end = process_time()

#print(t1_end - t1_start)
print(t2_end - t2_start)


## Monte Carlo simulation of torsion of circular shaft



In this example, a normal distribution with 0 mean is used to generate num_reps number of radii and num_reps number of internal torque values.  Think of this as making num_reps measurements of radii and internal torques.  

These radii and internal torques that are hence "synthetically" (or artificially or numerically) "measured" using random sampling may be thought to have a distribution because of some measurement uncertainty.

The Monte Carlo simulation allows for a sensitivity analysis.  i.e, is shear stress more sensitive to radius or cross section or the internal torque.  

It is assumed that we have a single cross-section with a single internal torque.

In [0]:
#Run this cell first!
import numpy as np #linear algebra and random number generation functions
import matplotlib.pyplot as plt #plotting
import pandas as pd #to create dataframes: tables of diverse types of data
import plotly.express as px #fancy, interactive plotting

### Initial data creation: Synthetic measurements

In [0]:
num_reps= 15 #number of data points
             #what do you observe if you increase the number of random samples?


avg_r = 0 # mean radius measurement, mm
std_dev_r = 0.05 #std. dev of radius measurement, mm
delta_r = np.random.normal(avg_r, std_dev_r, num_reps).round(2) #dr
#print(delta)
radius = 20 + delta_r #Synthetic or MonteCarlo radii

avg_T = 0 #N-mm
std_dev_T = 35 #N-mm
delta_T = np.random.normal(avg_T, std_dev_T, num_reps).round(2) #dT
torque = 1000 + delta_T #Synthetic or MonteCarlo Torques

### Dataframe creation with geometric parameters and shear stress

In [0]:
strength_df = pd.DataFrame({'radius (mm)': radius, 'delta_r (mm)': delta_r, 'Torque (N-mm)': torque})
strength_df["J (mm^4)"] = 0.5*np.pi*strength_df["radius (mm)"]**4
strength_df["tau (kPa)"] = strength_df["Torque (N-mm)"]*strength_df["radius (mm)"]/strength_df["J (mm^4)"]*1000

### Interactive plots to compare tau vs T, tau vs r

Is shear stress (tau) more sensitive to torque than it is to radius or cross section?  If so (or not), what quantitative evidence compels you to say?

In [0]:
px.scatter_3d(strength_df, x = "radius (mm)", y="Torque (N-mm)", z = "tau (kPa)",
              color = "tau (kPa)", opacity=0.8)

In [0]:
px.scatter_matrix(strength_df, 
           dimensions = ["tau (kPa)", "Torque (N-mm)", "delta_r (mm)", "radius (mm)"],
           color="tau (kPa)") #data points are coloured by shear stress values (in kPa)

# Determination of $T-\omega-P$ design enveol

In [0]:
'''
Initial values
'''
d = 0.1 #meter
J = np.pi*d**4/32 #polar modulus, meter^4
omega = 2400/60 #Revs per second 
tau_allow = 50*10**4 #Pascal

'''
MC loop
for torque
'''
@jit
def mc_torque(nSamples):
  return 150*np.random.random_sample((nSamples,))

@jit
def mc_omega(nSamples):
  return (2400/60)*np.random.random_sample((nSamples,))

nSamp=100;
df = pd.DataFrame({'torque': mc_torque(nSamp), 'omega (RPS)': mc_omega(nSamp)})
df['tau'] = df['torque']*0.5*d/J
df['power'] = df.apply(lambda x: 2*np.pi*x['omega (RPS)']*x['torque'], axis=1)
df['safe'] = df.apply(lambda x: 'Yes' if x['tau'] < tau_allow else 'No', axis=1)
df.head(5)

,torque,omega (RPS),tau,power,safe
0,149.602962,18.510566,761921.626802,17399.620191,No
1,106.442801,0.630094,542108.732579,421.406598,No
2,148.708167,23.664943,757364.477149,22111.599415,No
3,98.348190,27.604476,500883.217180,17057.907131,No
4,8.522984,2.371449,43407.199512,126.994633,Yes


In [0]:
px.scatter_3d(df, x='torque', y='omega (RPS)', z='power',
          color='safe')